<a href="https://colab.research.google.com/github/orlandokj/nlp-sentiment-analyzer/blob/master/TrabalhoNLTK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho NLTK Orlando Krause

In [ ]:
from nltk.tokenize import word_tokenize
import pandas as pd

In [ ]:
import nltk
nltk.download('popular')
nltk.download('vader_lexicon')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

# Foi feito um replace usando expressão regular para padronizar a separação entre texto e sentimento:
Expressao: ***,(negative|positive|sentiment)"?;;;;?;?;?***  
Replace: ***#;#$1***

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/imdb.csv", sep="#;#")
df.sentiment.value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


positive    25000
negative    25000
Name: sentiment, dtype: int64

# Dividimos o dataset em treino e teste

In [ ]:
from sklearn.model_selection import train_test_split
X=df.review.values
Y=df.sentiment.values
X_train, X_test, Y_train, Y_test= train_test_split(X, Y, test_size=0.3, stratify=Y)

# Definimos a função tokenizadora, usando stemmer e filtrando stopwords.

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re

stemmer = SnowballStemmer("english")
enStopWords = set(stopwords.words("english"))

def tokenize_text(text):
  words = word_tokenize(text.replace("<br />", "").replace(".", " ").replace("n't", ' not').replace("'ll", ' will'), "english")
  filteredWords = filter(lambda word: word not in enStopWords, map(lambda word: stemmer.stem(word.lower()), words))
  return list(filter(lambda word: re.search("[a-z']", word) is not None, filteredWords))



# Aqui demora uns 5 minutos para transformar os reviews em um vetor usando a nossa função *tokenizer*

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(tokenizer=tokenize_text)
vec.fit(X_train)
X_train_vector=vec.transform(X_train)
X_test_vector=vec.transform(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


# Treinamos o classificador com o vetor criado anteriormente

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_vector, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

# Realizamos a previsão com o dataset de testes e salvamos o resultado em um data frame junto com o Ground Truth(gt)

In [ ]:
lr_predict_result = lr.predict(X_test_vector)
result_df = pd.DataFrame(data={'gt': Y_test, 'lr': lr_predict_result})

# Aqui começa o sentiment analyzer do NLTK, consideramos positivo quando o score de positivo for maior que o score de negativo

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

def sia_classify(text):
  result = sia.polarity_scores(text)
  # Positivo se o score de positivo for maior que o score de negativo
  return 'positive' if result['pos'] > result['neg'] else 'negative'

result_df['sia'] = list(map(sia_classify, X_test))

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


# Comparando acurácias

In [ ]:
from sklearn.metrics import accuracy_score
lr_score = accuracy_score(result_df['gt'].values, result_df['lr'].values)
sia_score = accuracy_score(result_df['gt'].values, result_df['sia'].values)

print(f'Acurácia da regressão logística: {lr_score:.2f}')
print(f'Acurácia do SentimentIntensityAnalyzer: {sia_score:.2f}')

Acurácia da regressão logística: 0.89
Acurácia do SentimentIntensityAnalyzer: 0.69
